In [1]:
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
sys.path.append('script')
from script import dbconn
pgconn = dbconn.db_connection_psycopg()

In [3]:
# Fectching data from the postgreSql database and put the value on raw_df
raw_df = dbconn.db_read_table_psycopg(pgconn,'xdr_data')

c:\Users\o876\Desktop\User-Analytics-in-telecommunication-industry\script\dbconn.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = sqlio.read_sql_query(sql,pgconn)


# User Overview analysis

In [ ]:
# Identifying the top 10 handsets used by the customers.
# Count handset usage
handset_counts = raw_df.groupby(['Handset Manufacturer', 'Handset Type']).size().reset_index(name='Count')

# Sort the results
sorted_handsets = handset_counts.sort_values(by='Count', ascending=False)

# Select the top 10 handsets
top_10_handsets = sorted_handsets.head(10)

# Print the top 10 handsets
print(top_10_handsets)

In [ ]:
# Identifying Top 3 handset manufacturers
manufacturer_counts = raw_df['Handset Manufacturer'].value_counts().reset_index()
manufacturer_counts.columns = ['Handset Manufacturer', 'Count']

# Sort the results
sorted_manufacturers = manufacturer_counts.sort_values(by='Count', ascending=False)

# Select the top 3 handset manufacturers
top_3_manufacturers = sorted_manufacturers.head(3)

# Print the top 3 handset manufacturers
print(top_3_manufacturers)

In [ ]:
# Identify the top 5 handsets per top 3 handset manufacturer
# Count handset manufacturers and types
manufacturer_type_counts = raw_df.groupby(['Handset Manufacturer', 'Handset Type']).size().reset_index(name='Count')

# Sort the results within each manufacturer
sorted_manufacturer_types = manufacturer_type_counts.groupby('Handset Manufacturer').apply(lambda x: x.nlargest(5, 'Count')).reset_index(drop=True)

# Print the top 5 handsets per top 3 handset manufacturers
top_3_manufacturers = sorted_manufacturer_types['Handset Manufacturer'].unique()[:3]

for manufacturer in top_3_manufacturers:
    print(f"Top 5 handsets for {manufacturer}:")
    manufacturer_handsets = sorted_manufacturer_types[sorted_manufacturer_types['Handset Manufacturer'] == manufacturer]
    print(manufacturer_handsets)
    print()

In [ ]:
# Identifying users’ behaviour on those Applications / Social Media, Google, Email, Youtube, Netflix, Gaming, Other.   

In [ ]:
# Aggregate the number of xDR sessions per user
sessions_per_user = raw_df.groupby('MSISDN/Number')['Bearer Id'].count()
print(sessions_per_user)

In [ ]:
# Aggregate the session duration per user
session_duration_per_user = raw_df.groupby('MSISDN/Number')['Dur. (ms)'].sum()

# Display the result
print(session_duration_per_user)

In [ ]:
# Aggregate the total download (Dl) and upload data per user
total_data_per_user = raw_df.groupby('MSISDN/Number')[['Total DL (Bytes)', 'Total UL (Bytes)']].sum()

# Display the result
print(total_data_per_user)

In [ ]:
# Create a list of application columns
applications = ['Social Media', 'Google', 'Email', 'Youtube', 'Netflix', 'Gaming', 'Other']

# Aggregate the total data volume per user and application
total_data_per_user_app = raw_df.groupby('MSISDN/Number')[[col + ' DL (Bytes)' for col in applications] + [col + ' UL (Bytes)' for col in applications]].sum()

# Display the result
print(total_data_per_user_app)

In [ ]:
# Explanatory Data Analysis | EDA
# Treat all missing values and outliers in the dataset by replacing by the mean of the corresponding column.

In [4]:
raw_df.isna().sum()

Bearer Id                                      991
Start                                            1
Start ms                                         1
End                                              1
End ms                                           1
Dur. (ms)                                        1
IMSI                                           570
MSISDN/Number                                 1066
IMEI                                           572
Last Location Name                            1153
Avg RTT DL (ms)                              27829
Avg RTT UL (ms)                              27812
Avg Bearer TP DL (kbps)                          1
Avg Bearer TP UL (kbps)                          1
TCP DL Retrans. Vol (Bytes)                  88146
TCP UL Retrans. Vol (Bytes)                  96649
DL TP < 50 Kbps (%)                            754
50 Kbps < DL TP < 250 Kbps (%)                 754
250 Kbps < DL TP < 1 Mbps (%)                  754
DL TP > 1 Mbps (%)             

In [5]:
#percent of missing data
def percent_missing(df):
    # Calculate total number of cells in dataframe
    totalCells = np.product(df.shape)

    # Count number of missing values per column
    missingCount = df.isnull().sum()

    # Calculate total number of missing values
    totalMissing = missingCount.sum()

    # Calculate percentage of missing values
    percentageMissing = (totalMissing / totalCells) * 100

    print("The dataset contains", round(percentageMissing, 2), "%", "missing values.")

percent_missing(raw_df)

C:\Users\o876\AppData\Local\Temp\ipykernel_6540\748560962.py:17: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  percent_missing(raw_df)


The dataset contains 12.5 % missing values.


In [6]:
# Replace missing values with column mean for numeric columns
numeric_columns = raw_df.select_dtypes(include=[np.number]).columns
raw_df[numeric_columns] = raw_df[numeric_columns].fillna(raw_df[numeric_columns].mean())

# Replace missing values with column mode for non-numeric columns
non_numeric_columns = raw_df.select_dtypes(exclude=[np.number]).columns
raw_df[non_numeric_columns] = raw_df[non_numeric_columns].fillna(raw_df[non_numeric_columns].mode().iloc[0])

# Identify and replace outliers with column mean for numeric columns
for col in numeric_columns:
    z_scores = (raw_df[col] - raw_df[col].mean()) / raw_df[col].std()
    outliers = (z_scores > 3) | (z_scores < -3)
    raw_df.loc[outliers, col] = raw_df[col].mean()

# Verify missing values and outliers have been treated
missing_values_after_treatment = raw_df.isnull().sum()
print("Missing Values After Treatment:\n", missing_values_after_treatment)

Missing Values After Treatment:
 Bearer Id                                   0
Start                                       0
Start ms                                    0
End                                         0
End ms                                      0
Dur. (ms)                                   0
IMSI                                        0
MSISDN/Number                               0
IMEI                                        0
Last Location Name                          0
Avg RTT DL (ms)                             0
Avg RTT UL (ms)                             0
Avg Bearer TP DL (kbps)                     0
Avg Bearer TP UL (kbps)                     0
TCP DL Retrans. Vol (Bytes)                 0
TCP UL Retrans. Vol (Bytes)                 0
DL TP < 50 Kbps (%)                         0
50 Kbps < DL TP < 250 Kbps (%)              0
250 Kbps < DL TP < 1 Mbps (%)               0
DL TP > 1 Mbps (%)                          0
UL TP < 10 Kbps (%)                         0
1

In [ ]:
raw_df.isna().sum()

In [7]:
#Solving The rest of missing values
def fix_missing_ffill(df, col):
    df[col] = df[col].fillna(method='ffill')
    return df[col]

raw_df['Start'] = fix_missing_ffill(raw_df, 'Start')
raw_df['End'] = fix_missing_ffill(raw_df, 'End')
raw_df['Last Location Name'] = fix_missing_ffill(raw_df, 'Last Location Name')

missing_values = raw_df.isna().sum()
print(missing_values)

C:\Users\o876\AppData\Local\Temp\ipykernel_2612\3212002553.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[col] = df[col].fillna(method='ffill')


Bearer Id                                   0
Start                                       0
Start ms                                    0
End                                         0
End ms                                      0
Dur. (ms)                                   0
IMSI                                        0
MSISDN/Number                               0
IMEI                                        0
Last Location Name                          0
Avg RTT DL (ms)                             0
Avg RTT UL (ms)                             0
Avg Bearer TP DL (kbps)                     0
Avg Bearer TP UL (kbps)                     0
TCP DL Retrans. Vol (Bytes)                 0
TCP UL Retrans. Vol (Bytes)                 0
DL TP < 50 Kbps (%)                         0
50 Kbps < DL TP < 250 Kbps (%)              0
250 Kbps < DL TP < 1 Mbps (%)               0
DL TP > 1 Mbps (%)                          0
UL TP < 10 Kbps (%)                         0
10 Kbps < UL TP < 50 Kbps (%)     

In [ ]:
# Analyze the basic metrics (mean, median, etc) from the dataset
raw_df.describe()

In [ ]:
# Conduct a Non-Graphical Univariate Analysis by computing dispersion parameters for each quantitative variable.

quantitative_variables = []

# Iterate over each column in the dataset to find the quantitative variable
for column in raw_df.columns:   
    if raw_df[column].dtype in [int, float]:
        quantitative_variables.append(column)

# Solution: By calculating the range or Difference b/n max and min value in each variable
for column_name in quantitative_variables:
    column_data = raw_df[column_name]
    data_range = column_data.max() - column_data.min()
    print("Range of", column_name, ":", data_range)

In [ ]:
# Conduct a Graphical Univariate Analysis by identifying the most suitable plotting options for each variable and interpret your findings.
# column_name = 'Avg RTT DL (ms)'
clean_Data = raw_df.dropna()
column_names = clean_Data.columns

for column_name in column_names:
    column_data = clean_Data[column_name]    
    plt.hist(column_data, bins=10)
    plt.xlabel(column_name)
    plt.ylabel('Frequency')
    plt.title('Histogram of ' + column_name)
    plt.show()

In [ ]:
# Variable transformations
# Segment the users into top five decile classes based on the total duration for all sessions and compute the total data (DL+UL) per decile class. 

# Calculate the total duration for all sessions for each user
user_total_duration = raw_df.groupby('MSISDN/Number')['Dur. (ms)'].sum()

# Segment users into decile classes
user_deciles = pd.qcut(user_total_duration, q=10, labels=False, duplicates='drop')

# Compute the total data (DL+UL) per decile class
data_per_decile = raw_df.groupby(user_deciles)[['Total DL (Bytes)', 'Total UL (Bytes)']].sum().reset_index()

In [ ]:
# Correlation Analysis – compute a correlation matrix for the following variables and interpret your findings: Social Media data, Google data, Email data, Youtube data, Netflix data, Gaming data, Other data 
# Select the columns for correlation analysis
columns = [
    'Social Media DL (Bytes)', 'Social Media UL (Bytes)',
    'Google DL (Bytes)', 'Google UL (Bytes)',
    'Email DL (Bytes)', 'Email UL (Bytes)',
    'Youtube DL (Bytes)', 'Youtube UL (Bytes)',
    'Netflix DL (Bytes)', 'Netflix UL (Bytes)',
    'Gaming DL (Bytes)', 'Gaming UL (Bytes)',
    'Other DL (Bytes)', 'Other UL (Bytes)'
]

# Create a subset dataframe with the selected columns
subset_df = raw_df[columns]

# Compute the correlation matrix
correlation_matrix = subset_df.corr()

# Display the correlation matrix
print(correlation_matrix)

#  User Engagement analysis

In [ ]:
# Aggregate the above metrics per customer id (MSISDN) and report the top 10 customers per engagement metric

# Aggregate metrics per customer
aggregated_data = raw_df.groupby('MSISDN/Number').agg({
    'Bearer Id': 'nunique',  # Session frequency
    'Dur. (ms)': 'sum',  # Session duration
    'Total UL (Bytes)': 'sum',  # Upload traffic
    'Total DL (Bytes)': 'sum'  # Download traffic
}).reset_index()

# Rename the columns for better readability
aggregated_data.rename(columns={
    'Bearer Id': 'Session Frequency',
    'Dur. (ms)': 'Session Duration',
    'Total UL (Bytes)': 'Total Upload Traffic',
    'Total DL (Bytes)': 'Total Download Traffic'
}, inplace=True)

# Report the top 10 customers per engagement metric
top_10_frequency = aggregated_data.nlargest(10, 'Session Frequency')
top_10_duration = aggregated_data.nlargest(10, 'Session Duration')
top_10_upload_traffic = aggregated_data.nlargest(10, 'Total Upload Traffic')
top_10_download_traffic = aggregated_data.nlargest(10, 'Total Download Traffic')

# Display the results
print("Top 10 customers by Session Frequency:")
print(top_10_frequency)

print("\nTop 10 customers by Session Duration:")
print(top_10_duration)

print("\nTop 10 customers by Total Upload Traffic:")
print(top_10_upload_traffic)

print("\nTop 10 customers by Total Download Traffic:")
print(top_10_download_traffic)

In [ ]:
# Normalize each engagement metric and run a k-means (k=3) to classify customers in three groups of engagement. 
# Aggregate metrics per customer
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

aggregated_data = raw_df.groupby('MSISDN/Number').agg({
    'Bearer Id': 'nunique',  # Session frequency
    'Dur. (ms)': 'sum',  # Session duration
    'Total UL (Bytes)': 'sum',  # Upload traffic
    'Total DL (Bytes)': 'sum'  # Download traffic
}).reset_index()

# Normalize the engagement metrics
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(aggregated_data.iloc[:, 1:])  # Exclude customer ID

# Run k-means clustering
k = 3
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(normalized_data)

# Add the cluster labels to the aggregated data
aggregated_data['Cluster'] = kmeans.labels_

# Report the top customers per cluster
top_customers_per_cluster = []
for i in range(k):
    cluster_customers = aggregated_data[aggregated_data['Cluster'] == i].nlargest(10, 'Bearer Id')
    top_customers_per_cluster.append(cluster_customers)

# Display the results
for i, cluster_customers in enumerate(top_customers_per_cluster):
    print(f"\nTop 10 customers in Cluster {i+1}:")
    print(cluster_customers)

In [ ]:
# Compute the minimum, maximum, average & total non-normalized metrics for each cluster. 
# Interpret your results visually with accompanying text explaining your findings.

# Aggregate metrics per customer
aggregated_data = raw_df.groupby('MSISDN/Number').agg({
    'Bearer Id': 'nunique',  # Session frequency
    'Dur. (ms)': 'sum',  # Session duration
    'Total UL (Bytes)': 'sum',  # Upload traffic
    'Total DL (Bytes)': 'sum'  # Download traffic
}).reset_index()

# Normalize the engagement metrics
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(aggregated_data.iloc[:, 1:])  # Exclude customer ID

# Run k-means clustering
k = 3
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(normalized_data)

# Add the cluster labels to the aggregated data
aggregated_data['Cluster'] = kmeans.labels_

# Compute non-normalized metrics for each cluster
cluster_metrics = aggregated_data.groupby('Cluster').agg({
    'Bearer Id': ['min', 'max', 'mean', 'sum'],  # Session frequency
    'Dur. (ms)': ['min', 'max', 'mean', 'sum'],  # Session duration
    'Total UL (Bytes)': ['min', 'max', 'mean', 'sum'],  # Upload traffic
    'Total DL (Bytes)': ['min', 'max', 'mean', 'sum']  # Download traffic
})

# Display the results
print("Non-normalized metrics for each cluster:")
print(cluster_metrics)

In [ ]:
# Aggregate user total traffic per application and derive the top 10 most engaged users per application

# Extract the relevant columns for application traffic
app_columns = ['MSISDN/Number', 'Social Media DL (Bytes)', 'Google DL (Bytes)', 
               'Email DL (Bytes)', 'Youtube DL (Bytes)', 'Netflix DL (Bytes)', 
               'Gaming DL (Bytes)', 'Other DL (Bytes)']
app_traffic = raw_df[app_columns].copy()

# Rename the application columns for easier processing
app_traffic.columns = ['MSISDN/Number', 'Social Media', 'Google', 'Email', 'Youtube', 'Netflix', 'Gaming', 'Other']

# Melt the dataframe to combine all application columns into a single 'Application' column
app_traffic = app_traffic.melt(id_vars='MSISDN/Number', var_name='Application', value_name='Total Traffic')

# Aggregate total traffic per application for each user
app_traffic = app_traffic.groupby(['MSISDN/Number', 'Application'])['Total Traffic'].sum().reset_index()

# Derive the top 10 most engaged users per application
top_users_per_app = []
unique_apps = app_traffic['Application'].unique()

for app in unique_apps:
    top_users = app_traffic[app_traffic['Application'] == app].nlargest(10, 'Total Traffic')
    top_users_per_app.append(top_users)

# Display the results
for i, app in enumerate(unique_apps):
    print(f"\nTop 10 most engaged users for Application '{app}':")
    print(top_users_per_app[i])

In [ ]:
# Plot the top 3 most used applications using appropriate charts. 

# Extract the relevant columns for application traffic
app_columns = ['MSISDN/Number', 'Social Media DL (Bytes)', 'Google DL (Bytes)', 'Email DL (Bytes)', 'Youtube DL (Bytes)', 'Netflix DL (Bytes)', 'Gaming DL (Bytes)', 'Other DL (Bytes)']
app_traffic = raw_df[app_columns].copy()

# Rename the application columns for easier processing
app_traffic.columns = ['MSISDN/Number', 'Social Media', 'Google', 'Email', 'Youtube', 'Netflix', 'Gaming', 'Other']

# Melt the dataframe to combine all application columns into a single 'Application' column
app_traffic = app_traffic.melt(id_vars='MSISDN/Number', var_name='Application', value_name='Total Traffic')

# Aggregate total traffic per application
app_traffic = app_traffic.groupby('Application')['Total Traffic'].sum().reset_index()

# Sort applications by total traffic in descending order
app_traffic = app_traffic.sort_values('Total Traffic', ascending=False)

# Select the top 3 most used applications
top_3_apps = app_traffic.head(3)

# Plot the top 3 most used applications
plt.figure(figsize=(8, 6))
plt.bar(top_3_apps['Application'], top_3_apps['Total Traffic'])
plt.xlabel('Application')
plt.ylabel('Total Traffic')
plt.title('Top 3 Most Used Applications')
plt.show()

In [ ]:
# Using k-means clustering algorithm, group users in k engagement clusters based on the engagement metrics: 
# What is the optimized value of k (use elbow method for this)?

# Experience Analytics

In [ ]:
# Aggregate average TCP retransmission per customer
average_tcp_retransmission = raw_df.groupby('MSISDN/Number')['TCP DL Retrans. Vol (Bytes)'].mean()

# Print the aggregated results
print("Average TCP Retransmission per Customer:")
print(average_tcp_retransmission)

In [ ]:
# Aggregate average RTT per customer
average_rtt = raw_df.groupby('MSISDN/Number')['Avg RTT DL (ms)'].mean()

# Print the aggregated results
print("Average RTT per Customer:")
print(average_rtt)

In [ ]:
# Aggregate handset type per customer
handset_type = raw_df.groupby('MSISDN/Number')['Handset Type'].first()

# Print the aggregated results
print("Aggregate handset type per Customer:")
print(handset_type)

In [ ]:
# Aggregate average throughput per customer
average_throughput = raw_df.groupby('MSISDN/Number')['Avg Bearer TP DL (kbps)'].mean()

# Print the aggregated results
print("Average Throughput per Customer:")
print(average_throughput)

In [ ]:
# Compute & list 10 of the top, bottom and most frequent:

In [ ]:
# Get the top 10 TCP values
top_tcp_values = raw_df['TCP DL Retrans. Vol (Bytes)'].nlargest(10)

# Get the bottom 10 TCP values
bottom_tcp_values = raw_df['TCP DL Retrans. Vol (Bytes)'].nsmallest(10)

# Get the most frequent TCP values
most_frequent_tcp_values = raw_df['TCP DL Retrans. Vol (Bytes)'].value_counts().head(10)

# Print the results
print("Top 10 TCP Values:")
print(top_tcp_values)
print("\nBottom 10 TCP Values:")
print(bottom_tcp_values)
print("\nMost Frequent TCP Values:")
print(most_frequent_tcp_values)

In [ ]:
# Get the top 10 RTT values
top_rtt_values = raw_df['Avg RTT DL (ms)'].nlargest(10)

# Get the bottom 10 RTT values
bottom_rtt_values = raw_df['Avg RTT DL (ms)'].nsmallest(10)

# Get the most frequent RTT values
most_frequent_rtt_values = raw_df['Avg RTT DL (ms)'].value_counts().head(10)

# Print the results
print("Top 10 RTT Values:")
print(top_rtt_values)
print("\nBottom 10 RTT Values:")
print(bottom_rtt_values)
print("\nMost Frequent RTT Values:")
print(most_frequent_rtt_values)

In [ ]:
# Get the top 10 throughput values
top_throughput_values = raw_df['Avg Bearer TP DL (kbps)'].nlargest(10)

# Get the bottom 10 throughput values
bottom_throughput_values = raw_df['Avg Bearer TP DL (kbps)'].nsmallest(10)

# Get the most frequent throughput values
most_frequent_throughput_values = raw_df['Avg Bearer TP DL (kbps)'].value_counts().head(10)

# Print the results
print("Top 10 Throughput Values:")
print(top_throughput_values)
print("\nBottom 10 Throughput Values:")
print(bottom_throughput_values)
print("\nMost Frequent Throughput Values:")
print(most_frequent_throughput_values)

In [ ]:
# Compute the distribution of average throughput per handset type
avg_throughput_distribution = raw_df.groupby('Handset Type')['Avg Bearer TP DL (kbps)'].mean()

# Plot the distribution
plt.figure(figsize=(12, 6))
avg_throughput_distribution.plot(kind='bar')
plt.xlabel('Handset Type')
plt.ylabel('Average Throughput (kbps)')
plt.title('Distribution of Average Throughput per Handset Type')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Compute the average TCP retransmission view per handset type
avg_tcp_retransmission = raw_df.groupby('Handset Type')['TCP DL Retrans. Vol (Bytes)'].mean()

# Plot the average TCP retransmission view per handset type
plt.figure(figsize=(12, 6))
avg_tcp_retransmission.plot(kind='bar')
plt.xlabel('Handset Type')
plt.ylabel('Average TCP Retransmission')
plt.title('Average TCP Retransmission View per Handset Type')
plt.xticks(rotation=90)
plt.show()

In [ ]:
from sklearn.cluster import KMeans

# Select the relevant columns for clustering
data = raw_df[['Avg RTT DL (ms)', 'Avg Bearer TP DL (kbps)', 'TCP DL Retrans. Vol (Bytes)']]

# Perform k-means clustering
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(data)

# Add cluster labels to the dataset
raw_df['Cluster'] = kmeans.labels_

# Describe each cluster
cluster_descriptions = raw_df.groupby('Cluster').agg({
    'Avg RTT DL (ms)': 'mean',
    'Avg Bearer TP DL (kbps)': 'mean',
    'TCP DL Retrans. Vol (Bytes)': 'mean'
})

# Print the cluster descriptions
print("Cluster Descriptions:")
print(cluster_descriptions)

# Satisfaction Analysis

In [8]:
# Engagement score to each user. Consider the engagement score as the Euclidean distance between the user data point & 
# the less engaged cluster (use the first clustering for this) (Euclidean Distance)

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

engagement_columns = ['Social Media DL (Bytes)', 'Google DL (Bytes)', 'Email DL (Bytes)', 'Youtube DL (Bytes)', 'Netflix DL (Bytes)', 'Gaming DL (Bytes)', 'Other DL (Bytes)']
engagement_data = raw_df[engagement_columns]

# Perform clustering
kmeans = KMeans(n_clusters=3, random_state=42).fit(MinMaxScaler().fit_transform(engagement_data))
raw_df['Cluster'] = kmeans.predict(MinMaxScaler().fit_transform(engagement_data))

# Compute engagement scores
raw_df['Engagement Score'] = kmeans.transform(MinMaxScaler().fit_transform(engagement_data)).min(axis=1)

# Display the top 10 users with the lowest engagement scores
top_10_least_engaged_users = raw_df.nsmallest(10, 'Engagement Score')
print("Top 10 least engaged users:")
print(top_10_least_engaged_users[['MSISDN/Number', 'Cluster', 'Engagement Score']])

c:\Users\o876\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Top 10 least engaged users:
        MSISDN/Number  Cluster  Engagement Score
16009    3.360985e+10        1          0.068658
148295   3.369948e+10        2          0.144754
117786   3.366856e+10        2          0.156107
71508    3.366376e+10        0          0.161020
115538   3.365013e+10        0          0.161214
35951    3.366480e+10        1          0.164404
34585    3.366612e+10        1          0.164657
127173   3.366355e+10        2          0.168642
71466    3.376233e+10        2          0.168886
34146    3.366444e+10        1          0.169006


In [10]:
# Experience score to each user. Consider the experience score as the Euclidean distance between the user data point & the worst experience’s cluster. 

engagement_columns = ['Social Media DL (Bytes)', 'Google DL (Bytes)', 'Email DL (Bytes)', 'Youtube DL (Bytes)', 'Netflix DL (Bytes)', 'Gaming DL (Bytes)', 'Other DL (Bytes)']
engagement_data = raw_df[engagement_columns]

# Perform clustering
kmeans = KMeans(n_clusters=3, random_state=42).fit(MinMaxScaler().fit_transform(engagement_data))
raw_df['Cluster'] = kmeans.predict(MinMaxScaler().fit_transform(engagement_data))

# Compute the worst experience cluster
worst_experience_cluster = np.argmin(kmeans.inertia_)
worst_experience_cluster_centroid = kmeans.cluster_centers_[worst_experience_cluster]

# Compute experience scores
raw_df['Experience Score'] = kmeans.transform(MinMaxScaler().fit_transform(engagement_data)).min(axis=1)

# Display the top 10 users with the worst experience scores
top_10_worst_experience_users = raw_df.nsmallest(10, 'Experience Score')
print("Top 10 users with the worst experience:")
print(top_10_worst_experience_users[['MSISDN/Number', 'Cluster', 'Experience Score']])

c:\Users\o876\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Top 10 users with the worst experience:
        MSISDN/Number  Cluster  Experience Score
16009    3.360985e+10        1          0.068658
148295   3.369948e+10        2          0.144754
117786   3.366856e+10        2          0.156107
71508    3.366376e+10        0          0.161020
115538   3.365013e+10        0          0.161214
35951    3.366480e+10        1          0.164404
34585    3.366612e+10        1          0.164657
127173   3.366355e+10        2          0.168642
71466    3.376233e+10        2          0.168886
34146    3.366444e+10        1          0.169006


In [11]:
# Consider the average of both engagement & experience scores as  the satisfaction score & report the top 10 satisfied customer

# Calculate engagement scores
engagement_data = raw_df[['Social Media DL (Bytes)', 'Google DL (Bytes)', 'Email DL (Bytes)', 'Youtube DL (Bytes)', 'Netflix DL (Bytes)', 'Gaming DL (Bytes)', 'Other DL (Bytes)']]
engagement_scores = KMeans(n_clusters=3, random_state=42).fit_transform(MinMaxScaler().fit_transform(engagement_data)).min(axis=1)

# Calculate experience scores
experience_scores = KMeans(n_clusters=3, random_state=42).fit_transform(MinMaxScaler().fit_transform(engagement_data)).min(axis=1)

# Calculate satisfaction scores as the average of engagement and experience scores
satisfaction_scores = (engagement_scores + experience_scores) / 2

# Add satisfaction scores to the DataFrame
raw_df['Satisfaction Score'] = satisfaction_scores

# Report the top 10 satisfied customers
top_10_satisfied_customers = raw_df.nlargest(10, 'Satisfaction Score')
print("Top 10 satisfied customers:")
print(top_10_satisfied_customers[['MSISDN/Number', 'Satisfaction Score']])

c:\Users\o876\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\o876\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Top 10 satisfied customers:
        MSISDN/Number  Satisfaction Score
124713   3.369943e+10            1.129860
43612    3.363233e+10            1.128138
38648    3.367117e+10            1.120422
35417    3.366736e+10            1.118322
143967   3.378923e+10            1.113666
84003    3.365011e+10            1.110356
132887   3.366400e+10            1.106980
1568     3.375586e+10            1.102493
30080    3.366728e+10            1.102431
110543   3.366297e+10            1.102149


In [12]:
# Build a regression model of your choice to predict the satisfaction score of a customer. 

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Prepare the features and target variables
features = raw_df[['Social Media DL (Bytes)', 'Google DL (Bytes)', 'Email DL (Bytes)', 'Youtube DL (Bytes)', 'Netflix DL (Bytes)', 'Gaming DL (Bytes)', 'Other DL (Bytes)']]
target = raw_df['Satisfaction Score']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Create and train the Random Forest regression model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the model's performance metrics
print("Mean Squared Error (MSE):", mse)
print("R-squared (R2) Score:", r2)

Mean Squared Error (MSE): 0.0010827734241375883
R-squared (R2) Score: 0.9365999833474234


In [13]:
# Run a k-means (k=2) on the engagement & the experience score.

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

# Prepare the engagement and experience scores
engagement_columns = ['Social Media DL (Bytes)', 'Google DL (Bytes)', 'Email DL (Bytes)', 'Youtube DL (Bytes)', 'Netflix DL (Bytes)', 'Gaming DL (Bytes)', 'Other DL (Bytes)']
engagement_data = raw_df[engagement_columns]

# Scale the engagement data
scaler = MinMaxScaler().fit(engagement_data)
scaled_engagement_data = scaler.transform(engagement_data)

# Run k-means clustering with k=2
kmeans = KMeans(n_clusters=2, random_state=42).fit(scaled_engagement_data)

# Get the cluster labels
cluster_labels = kmeans.labels_

# Add the cluster labels to the DataFrame
raw_df['Engagement Cluster'] = cluster_labels

# Compute experience scores
experience_scores = kmeans.transform(scaled_engagement_data).min(axis=1)

# Add experience scores to the DataFrame
raw_df['Experience Score'] = experience_scores

# Display the clusters and experience scores
print("Engagement Clusters:")
print(raw_df['Engagement Cluster'].value_counts())
print("\nExperience Scores:")
print(raw_df['Experience Score'])

c:\Users\o876\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Engagement Clusters:
Engagement Cluster
0    75361
1    74640
Name: count, dtype: int64

Experience Scores:
0         0.726574
1         0.623798
2         0.552089
3         0.758693
4         0.439322
            ...   
149996    0.980796
149997    0.604966
149998    0.649726
149999    0.708617
150000    0.249494
Name: Experience Score, Length: 150001, dtype: float64


In [14]:
# Aggregate the average satisfaction & experience score per cluster. 
cluster_agg = raw_df.groupby('Engagement Cluster').agg({'Satisfaction Score': 'mean', 'Experience Score': 'mean'})
print(cluster_agg)

                    Satisfaction Score  Experience Score
Engagement Cluster                                      
0                             0.682653          0.710019
1                             0.682582          0.709398
